In [1]:
import os
import string
import re
import numpy as np
import pandas as p
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import nltk

/home/avinash/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
reviews = []
for root, dirs, files in os.walk("/home/avinash/Downloads/Reviews/"):
        for f in files:
            if os.path.splitext(f)[-1] == ".txt":
                with open(os.path.join(root, f), "r") as infile:
                    print(os.path.join(root, f))
                    for line in infile:
                        reviews.append(line)

/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_knickerbocker_10.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_homewood_8.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_swissotel_2.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_knickerbocker_4.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_sheraton_18.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_swissotel_11.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_knickerbocker_6.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_sheraton_11.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_sheraton_3.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web/fold4/t_swissotel_20.txt
/home/avinash/Downloads/Reviews/negative_polarity/truthful_from_Web

/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_hilton_1.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_hilton_12.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_james_20.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_monaco_8.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_monaco_13.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_james_17.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_sofitel_20.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_sofitel_11.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_monaco_3.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fold1/d_sofitel_1.txt
/home/avinash/Downloads/Reviews/negative_polarity/deceptive_from_MTurk/fo

In [3]:
df = p.DataFrame(reviews, columns=['review'])

index=np.arange(1600)
df['index'] = index

for label, row in df.iterrows():
    if (int(row['index']) < 400):
        df.loc[label, 'label']= 'truthful'
    if (int(row['index'])>400 and int(row['index'])<800):
        df.loc[label, 'label']= 'deceptive'
    if (int(row['index'])>800 and int(row['index'])<1200 ):
        df.loc[label, 'label']= 'truthful'
    if(int(row['index'])>1200 and int(row['index'])<1600 ):
        df.loc[label, 'label']= 'deceptive'
df

,review,index,label
0,"Our suite at the Knickerbocker are quite nice,...",0,truthful
1,"This hotel had a great location, but you can d...",1,truthful
2,Was one of the worst travel experiences of qui...,2,truthful
3,From checkin to departure a poorly run hotel. ...,3,truthful
4,"Generally speaking, nothing bad about that pla...",4,truthful
5,My wife and I frequently stay downtown for a n...,5,truthful
6,"Before booking this hotel, I did read the revi...",6,truthful
7,We had reservations (guaranteed by credit card...,7,truthful
8,"We stayed here for one night, and found it a h...",8,truthful
9,"Swisses Chicago: Nice, but lacking in some pre...",9,truthful


In [4]:
STOPWORDS = """
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves
"""
STOPWORDS = frozenset(w for w in STOPWORDS.split() if w)

def any2unicode(text, encoding='utf8', errors='strict'):
    if isinstance(text, str):
        return text
    return str(text, encoding, errors=errors)

def remove_stopwords(s):
    s = any2unicode(s)
    return " ".join(w for w in s.split() if w not in STOPWORDS)


RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)
def strip_punctuation(s):
    s = any2unicode(s)
    return RE_PUNCT.sub(" ", s)


# unicode.translate cannot delete characters like str can
strip_punctuation2 = strip_punctuation

RE_TAGS = re.compile(r"<([^>]+)>", re.UNICODE)
def strip_tags(s):
    s = any2unicode(s)
    return RE_TAGS.sub("",s)


def strip_short(s, minsize=3):
    s = any2unicode(s)
    return " ".join(e for e in s.split() if len(e) >= minsize)


RE_NUMERIC = re.compile(r"[0-9]+", re.UNICODE)
def strip_numeric(s):
    s = any2unicode(s)
    return RE_NUMERIC.sub("", s)


RE_NONALPHA = re.compile(r"\W", re.UNICODE)
def strip_non_alphanum(s):
    s = any2unicode(s)
    return RE_NONALPHA.sub(" ", s)


RE_WHITESPACE = re.compile(r"(\s)+", re.UNICODE)
def strip_multiple_whitespaces(s):
    s = any2unicode(s)
    return RE_WHITESPACE.sub(" ", s)


RE_AL_NUM = re.compile(r"([a-z]+)([0-9]+)", flags=re.UNICODE)
RE_NUM_AL = re.compile(r"([0-9]+)([a-z]+)", flags=re.UNICODE)
def split_alphanum(s):
    s = any2unicode(s)
    s = RE_AL_NUM.sub(r"\1 \2", s)
    return RE_NUM_AL.sub(r"\1 \2", s)


DEFAULT_FILTERS = [lambda x: x.lower(), strip_tags, strip_numeric, strip_short, strip_non_alphanum, strip_multiple_whitespaces, split_alphanum, remove_stopwords]

def preprocess_string(s, filters=DEFAULT_FILTERS):
    s = any2unicode(s)
    for f in filters:
        s = f(s)
    return s

In [5]:
preprocess = df.apply(lambda row: preprocess_string(str(row['review'])), axis=1)
print (preprocess)

0       suite knickerbocker nice behavior attitude sta...
1       hotel great location better money hallways sme...
2       worst travel experiences time arrived construc...
3       checkin departure poorly run hotel took minute...
4       generally speaking bad place cleaning issue ch...
5       wife frequently stay downtown night city hotel...
6       booking hotel read reviews concerned ended boo...
7       reservations guaranteed credit card nights pas...
8       stayed night hotel concentrates conferences no...
9       swisses chicago nice lacking pretty basic expe...
10      stayed april nights hesitated selecting sherat...
11      booked hotel stay yahoo requested room beds go...
12      conference bed bugs bug bites itches like hell...
13      fiance traveled chicago time december happy ho...
14      historic feel hotel strong influence choice bo...
15      lobby homewood right hall kitchen dining room ...
16      reading good reviews booked night stay checked...
17      ideal 

In [6]:
review_pos = nltk.pos_tag(preprocess)
print (review_pos)

[('suite knickerbocker nice behavior attitude staff abysmal member staff entered room mid day unannounced apologize realizing failed verify present tremendous problem young woman desk replied want tone flippant teenager began suspect severe deficiency manners evident staff fast forward evening following lovely meal premises chose turn bit early discover despite best intentions bass party downstairs audible room acting prudently restraint decided phone desk inquire room inadvertently annexed party downstairs met curt rude response sean desk informed party greater priority hotel family dictated music set certain volume hand appreciated honesty truly preferred pretended care guests chose stay property options evening ve decided stay try watch time given circumstances staff easily negative situations arose quick succession alarmed learn slept prior night possible gang related shooting took place mere blocks property wouldn t concern like nearby hotels knickerbocker chose enlist type securi

In [7]:
new_list = []
for pos in review_pos:
    new_list.append(' '.join(pos))
df_review_pos = p.DataFrame(new_list).squeeze()
X = df_review_pos
y = df.label
X = X.fillna(method='ffill')
y = y.fillna(method='ffill')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [8]:
tf_vect = TfidfVectorizer(ngram_range=(1,3), lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(X_train)
X_test_tf = tf_vect.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

#making pedictions
y_predict = mnb.predict(X_test_tf)

In [9]:
print(metrics.accuracy_score(y_test, y_predict))
print (confusion_matrix(y_test, y_predict))
print (classification_report(y_test, y_predict))

0.871875
[[154   5]
 [ 36 125]]
             precision    recall  f1-score   support

  deceptive       0.81      0.97      0.88       159
   truthful       0.96      0.78      0.86       161

avg / total       0.89      0.87      0.87       320

